In [1]:
import pandas as pd

import numpy as np

import string

from nltk.stem import PorterStemmer

from nltk.tokenize import RegexpTokenizer

from nltk.corpus import stopwords
from statsmodels.graphics.mosaicplot import mosaic

import matplotlib.pyplot as plt

import torch
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import Dataset

In [2]:
!pip install modelscope

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 52.1 MB/s eta 0:00:00


In [3]:
from modelscope import snapshot_download
model_id = snapshot_download("deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B")

2025-03-11 23:00:40,059 - modelscope - INFO - Got 9 files, start to download ...


Processing 9 items:   0%|          | 0.00/9.00 [00:00<?, ?it/s]

2025-03-11 23:02:25,219 - modelscope - INFO - Download model 'deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B' successfully.
2025-03-11 23:02:25,220 - modelscope - INFO - Creating symbolic link [/root/.cache/modelscope/hub/models/deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B].


In [4]:
from transformers import pipeline

In [5]:
deepseek_pipeline = pipeline(
    "text-generation",
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B",
    batch_size=16,
    device="cuda",
    model_kwargs={"torch_dtype": "auto"}  
)


config.json:   0%|          | 0.00/679 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.55G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/3.07k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

Device set to use cuda


In [6]:
test_filepath_csv = '/kaggle/input/nli-data/new_test_all.csv'

In [7]:
test_df = pd.read_csv(test_filepath_csv)
test_df

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,Unnamed: 0,premise,hypothesis,gold_label,type
0,9616.0,"If Charlotte passes her driving test, her pare...",Charlotte has parents.,E,type5_possessive
1,9617.0,It's not the case that if Charlotte passes her...,Charlotte has parents.,E,type5_possessive
2,9618.0,Sam believes that if Charlotte passes her driv...,Charlotte has parents.,E,type5_possessive
3,9619.0,Is it true that if Charlotte passes her drivin...,Charlotte has parents.,E,type5_possessive
4,9628.0,"If Lily takes the train to the city, her frien...",Lily has a friend.,E,type5_possessive
...,...,...,...,...,...
2362,NaN,"If Sophia decides to attend the concert, his b...",Sophia has a close friend.,N,type5_possessive
2363,NaN,It's not the case that if Sophia decides to at...,Sophia has a close friend.,N,type5_possessive
2364,NaN,Sam believes that if Sophia decides to attend ...,Sophia has a close friend.,N,type5_possessive
2365,NaN,Is it true that if Sophia decides to attend th...,Sophia has a close friend.,N,type5_possessive


In [8]:
def create_message(row):
    return {
        "role": "user",
        "content": f'''
        Does the premise entail the hypothesis? 
        Output E if the premise does entail the hypothesis, or N if it does not. 
        Output C if the hypothesis contradicts the premise. Do not output anything other than E, N, or C.
        Premise: {row["premise"]}
        Hypothesis: {row["hypothesis"]}
        Output:
        '''
    }

In [9]:
# Split the DataFrame into two halves
half = len(test_df) // 2
test_df1 = test_df[:half]
test_df2 = test_df[half:]

In [10]:
test_df1

,Unnamed: 0,premise,hypothesis,gold_label,type
0,9616.0,"If Charlotte passes her driving test, her pare...",Charlotte has parents.,E,type5_possessive
1,9617.0,It's not the case that if Charlotte passes her...,Charlotte has parents.,E,type5_possessive
2,9618.0,Sam believes that if Charlotte passes her driv...,Charlotte has parents.,E,type5_possessive
3,9619.0,Is it true that if Charlotte passes her drivin...,Charlotte has parents.,E,type5_possessive
4,9628.0,"If Lily takes the train to the city, her frien...",Lily has a friend.,E,type5_possessive
...,...,...,...,...,...
1178,2274.0,Is it true that if Amy ever bought a smartphon...,Amy has bought an iPhone.,N,type3_again
1179,2280.0,It's not the case that if the president ever s...,The president has signed the ceasefire agreement.,N,type3_again
1180,2285.0,It's not the case that if the president ever p...,The president has participated in the Rolling ...,N,type3_again
1181,2287.0,It's not the case that if the old man ever fed...,The old man has fed a snow leopard.,N,type3_again


In [11]:
test_df1['message'] = test_df1.apply(create_message, axis=1)
test_df1

<ipython-input-11-a3a8a09df9c9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df1['message'] = test_df1.apply(create_message, axis=1)


,Unnamed: 0,premise,hypothesis,gold_label,type,message
0,9616.0,"If Charlotte passes her driving test, her pare...",Charlotte has parents.,E,type5_possessive,"{'role': 'user', 'content': ' Does the..."
1,9617.0,It's not the case that if Charlotte passes her...,Charlotte has parents.,E,type5_possessive,"{'role': 'user', 'content': ' Does the..."
2,9618.0,Sam believes that if Charlotte passes her driv...,Charlotte has parents.,E,type5_possessive,"{'role': 'user', 'content': ' Does the..."
3,9619.0,Is it true that if Charlotte passes her drivin...,Charlotte has parents.,E,type5_possessive,"{'role': 'user', 'content': ' Does the..."
4,9628.0,"If Lily takes the train to the city, her frien...",Lily has a friend.,E,type5_possessive,"{'role': 'user', 'content': ' Does the..."
...,...,...,...,...,...,...
1178,2274.0,Is it true that if Amy ever bought a smartphon...,Amy has bought an iPhone.,N,type3_again,"{'role': 'user', 'content': ' Does the..."
1179,2280.0,It's not the case that if the president ever s...,The president has signed the ceasefire agreement.,N,type3_again,"{'role': 'user', 'content': ' Does the..."
1180,2285.0,It's not the case that if the president ever p...,The president has participated in the Rolling ...,N,type3_again,"{'role': 'user', 'content': ' Does the..."
1181,2287.0,It's not the case that if the old man ever fed...,The old man has fed a snow leopard.,N,type3_again,"{'role': 'user', 'content': ' Does the..."


In [12]:
test_df2

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,Unnamed: 0,premise,hypothesis,gold_label,type
1183,2294.0,Sam believes that if John ever trusted a condu...,John has trusted Leonard Bernstein.,N,type3_again
1184,2296.0,"Sam believes that if Amy ever pet a parrot, sh...",Amy has pet a Macaw.,N,type3_again
1185,2297.0,It's not the case that if Carlos ever watched ...,Carlos has watched Long Walk to Freedom.,N,type3_again
1186,2299.0,"If the woman ever studied a genre, she didn't ...",The woman has studied science fiction.,N,type3_again
1187,2306.0,Sam believes that if the professor ever used a...,The professor has used Pointillism.,N,type3_again
...,...,...,...,...,...
2362,NaN,"If Sophia decides to attend the concert, his b...",Sophia has a close friend.,N,type5_possessive
2363,NaN,It's not the case that if Sophia decides to at...,Sophia has a close friend.,N,type5_possessive
2364,NaN,Sam believes that if Sophia decides to attend ...,Sophia has a close friend.,N,type5_possessive
2365,NaN,Is it true that if Sophia decides to attend th...,Sophia has a close friend.,N,type5_possessive


In [13]:
def classify_batch(batch):
    messages = batch['few_shot_message']
    outputs = [deepseek_pipeline(message, max_new_tokens=2000, do_sample=True, temperature=0.6, top_p=0.9) for message in messages]
    # Extract the final character (E, N, or C) from each output
    results = [output[0]["generated_text"][-1]['content'][-1] for output in outputs]
    return {"Predicted - Zero-Shot DeepSeek": results}

## Few-Shot

In [14]:
def create_fewshot_message(row):
    messages = [
        # type 1 again
         {"role": "user", "content": f'''
        Consider the truth value of the premise.  If the premise is true, does it necessarily mean that the hypothesis must also be true?
        Output E if the hypothesis must always be true
        Output C if the hypothesis must always be false
        Output N if the hypothesis may be either true or false
        Do not output anything other than E, C, or N
        Premise: Sam believes that if the boss rewarded a good behavior by Nava, they'll never reward a good behavior by Nava again.
        Hypothesis: The boss rewarded a good behavior by Nava.
        Output:
        '''},
        {"role": "assistant", "content": "N"},
        # type 1 possessive
        {"role": "user", "content": f'''
        Consider the truth value of the premise.  If the premise is true, does it necessarily mean that the hypothesis must also be true?
        Output E if the hypothesis must always be true
        Output C if the hypothesis must always be false
        Output N if the hypothesis may be either true or false
        Do not output anything other than E, C, or N
        Premise: If Alexander has a bright roommate, he'll rest with his bright roommate.
        Hypothesis: Alexander has a bright roommate..
        Output:
        '''},
        {"role": "assistant", "content": "N"},
        # type 2 again - negated
        {"role": "user", "content": f'''
        Consider the truth value of the premise.  If the premise is true, does it necessarily mean that the hypothesis must also be true?
        Output E if the hypothesis must always be true
        Output C if the hypothesis must always be false
        Output N if the hypothesis may be either true or false
        Do not output anything other than E, C, or N
        Premise: It's not the case that if Curtis reports on this issue, he'll never report on an issue again.
        Hypothesis: Curtis has reported on an issue before.
        Output:
        '''},
        {"role": "assistant", "content": "N"},
        # type 2 possessive - believes
        {"role": "user", "content": f'''
        Consider the truth value of the premise.  If the premise is true, does it necessarily mean that the hypothesis must also be true?
        Output E if the hypothesis must always be true
        Output C if the hypothesis must always be false
        Output N if the hypothesis may be either true or false
        Do not output anything other than E, C, or N
        Premise: Sam believes that if Marley has a curious friend, he gives his friend a ride home.
        Hypothesis: Marley has a friend.
        Output:
        '''},
        {"role": "assistant", "content": "N"},
        # type 3 again - believe
        {"role": "user", "content": f'''
        Consider the truth value of the premise.  If the premise is true, does it necessarily mean that the hypothesis must also be true?
        Output E if the hypothesis must always be true
        Output C if the hypothesis must always be false
        Output N if the hypothesis may be either true or false
        Do not output anything other than E, C, or N
        Premise: Sam believes that if Alice ever drank a cocktail, she didn't drink martini again.
        Hypothesis: Alice has drunk a martini.
        Output:
        '''},
        {"role": "assistant", "content": "N"},
        # type 3 possessive - believe
        {"role": "user", "content": f'''
        Consider the truth value of the premise.  If the premise is true, does it necessarily mean that the hypothesis must also be true?
        Output E if the hypothesis must always be true
        Output C if the hypothesis must always be false
        Output N if the hypothesis may be either true or false
        Do not output anything other than E, C, or N
        Premise: If Meredith has video games, she will play her immersive video game.
        Hypothesis: Meredith has an immersive video game.
        Output:
        '''},
        {"role": "assistant", "content": "N"},
        # type 4 - entailment
        {"role": "user", "content": f'''
        Consider the truth value of the premise.  If the premise is true, does it necessarily mean that the hypothesis must also be true?
        Output E if the hypothesis must always be true
        Output C if the hypothesis must always be false
        Output N if the hypothesis may be either true or false
        Do not output anything other than E, C, or N
        Premise: If Matt is a scuba diver, he'll bring his wetsuit.
        Hypothesis: Matt has a wetsuit.
        Output:
        '''},
        {"role": "assistant", "content": "E"},
        # type 4 - neutral
        {"role": "user", "content": f'''
        Consider the truth value of the premise.  If the premise is true, does it necessarily mean that the hypothesis must also be true?
        Output E if the hypothesis must always be true
        Output C if the hypothesis must always be false
        Output N if the hypothesis may be either true or false
        Do not output anything other than E, C, or N
        Premise: If Tina has a helicopter, I wonder how low Tina's helicopter hovers.
        Hypothesis: Tina has a helicopter.
        Output:
        '''},
        {"role": "assistant", "content": "N"},
        # type 5 again
        {"role": "user", "content": f'''
        Consider the truth value of the premise.  If the premise is true, does it necessarily mean that the hypothesis must also be true?
        Output E if the hypothesis must always be true
        Output C if the hypothesis must always be false
        Output N if the hypothesis may be either true or false
        Do not output anything other than E, C, or N
        Premise: If Emily passes her driving test, she'll never read a book by Dan Brown again.
        Hypothesis: Emily has read a book by Dan Brown before.
        Output:
        '''},
        {"role": "assistant", "content": "E"},
        # type 5 possessive
        {"role": "user", "content": f'''
        Consider the truth value of the premise.  If the premise is true, does it necessarily mean that the hypothesis must also be true?
        Output E if the hypothesis must always be true
        Output C if the hypothesis must always be false
        Output N if the hypothesis may be either true or false
        Do not output anything other than E, C, or N
        Premise: If Margaret's flight arrives on time, her friend will be waiting for her at the airport.
        Hypothesis: Margaret has a friend.
        Output:
        '''},
        {"role": "assistant", "content": "E"},
        {"role": "user", "content": f'''
        Consider the truth value of the premise.  If the premise is true, does it necessarily mean that the hypothesis must also be true?
        Output E if the hypothesis must always be true
        Output C if the hypothesis must always be false
        Output N if the hypothesis may be either true or false
        Do not output anything other than E, C, or N
        Premise: {row["premise"]}
        Hypothesis: {row["hypothesis"]}
        Output:
        '''},
    ]
    return messages

In [15]:
test_df1['few_shot_message'] = test_df1.apply(create_fewshot_message, axis=1)
test_df1

<ipython-input-15-99da5b0d093b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df1['few_shot_message'] = test_df1.apply(create_fewshot_message, axis=1)


,Unnamed: 0,premise,hypothesis,gold_label,type,message,few_shot_message
0,9616.0,"If Charlotte passes her driving test, her pare...",Charlotte has parents.,E,type5_possessive,"{'role': 'user', 'content': ' Does the...","[{'role': 'user', 'content': ' Conside..."
1,9617.0,It's not the case that if Charlotte passes her...,Charlotte has parents.,E,type5_possessive,"{'role': 'user', 'content': ' Does the...","[{'role': 'user', 'content': ' Conside..."
2,9618.0,Sam believes that if Charlotte passes her driv...,Charlotte has parents.,E,type5_possessive,"{'role': 'user', 'content': ' Does the...","[{'role': 'user', 'content': ' Conside..."
3,9619.0,Is it true that if Charlotte passes her drivin...,Charlotte has parents.,E,type5_possessive,"{'role': 'user', 'content': ' Does the...","[{'role': 'user', 'content': ' Conside..."
4,9628.0,"If Lily takes the train to the city, her frien...",Lily has a friend.,E,type5_possessive,"{'role': 'user', 'content': ' Does the...","[{'role': 'user', 'content': ' Conside..."
...,...,...,...,...,...,...,...
1178,2274.0,Is it true that if Amy ever bought a smartphon...,Amy has bought an iPhone.,N,type3_again,"{'role': 'user', 'content': ' Does the...","[{'role': 'user', 'content': ' Conside..."
1179,2280.0,It's not the case that if the president ever s...,The president has signed the ceasefire agreement.,N,type3_again,"{'role': 'user', 'content': ' Does the...","[{'role': 'user', 'content': ' Conside..."
1180,2285.0,It's not the case that if the president ever p...,The president has participated in the Rolling ...,N,type3_again,"{'role': 'user', 'content': ' Does the...","[{'role': 'user', 'content': ' Conside..."
1181,2287.0,It's not the case that if the old man ever fed...,The old man has fed a snow leopard.,N,type3_again,"{'role': 'user', 'content': ' Does the...","[{'role': 'user', 'content': ' Conside..."


In [16]:
test_ds1 = Dataset.from_pandas(test_df1)
test_ds1

Dataset({
    features: ['Unnamed: 0', 'premise', 'hypothesis', 'gold_label', 'type', 'message', 'few_shot_message'],
    num_rows: 1183
})

In [17]:
# subset = test_ds1.select(range(8))
# subset = subset.map(classify_batch,batched=True,batch_size=8)
# subset_result_df = subset.to_pandas()
# subset_result_df.to_csv('subset_results.csv')
# subset_result_df

In [18]:
results = test_ds1.map(classify_batch,batched=True,batch_size=8)
result_df = results.to_pandas()
result_df.to_csv('deepseek_test_df1_fewshot_results.csv')
result_df

Map:   0%|          | 0/1183 [00:00<?, ? examples/s]

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


,Unnamed: 0,premise,hypothesis,gold_label,type,message,few_shot_message,Predicted - Zero-Shot DeepSeek
0,9616.0,"If Charlotte passes her driving test, her pare...",Charlotte has parents.,E,type5_possessive,{'content': ' Does the premise entail ...,[{'content': ' Consider the truth valu...,N
1,9617.0,It's not the case that if Charlotte passes her...,Charlotte has parents.,E,type5_possessive,{'content': ' Does the premise entail ...,[{'content': ' Consider the truth valu...,N
2,9618.0,Sam believes that if Charlotte passes her driv...,Charlotte has parents.,E,type5_possessive,{'content': ' Does the premise entail ...,[{'content': ' Consider the truth valu...,N
3,9619.0,Is it true that if Charlotte passes her drivin...,Charlotte has parents.,E,type5_possessive,{'content': ' Does the premise entail ...,[{'content': ' Consider the truth valu...,N
4,9628.0,"If Lily takes the train to the city, her frien...",Lily has a friend.,E,type5_possessive,{'content': ' Does the premise entail ...,[{'content': ' Consider the truth valu...,N
...,...,...,...,...,...,...,...,...
1178,2274.0,Is it true that if Amy ever bought a smartphon...,Amy has bought an iPhone.,N,type3_again,{'content': ' Does the premise entail ...,[{'content': ' Consider the truth valu...,N
1179,2280.0,It's not the case that if the president ever s...,The president has signed the ceasefire agreement.,N,type3_again,{'content': ' Does the premise entail ...,[{'content': ' Consider the truth valu...,N
1180,2285.0,It's not the case that if the president ever p...,The president has participated in the Rolling ...,N,type3_again,{'content': ' Does the premise entail ...,[{'content': ' Consider the truth valu...,N
1181,2287.0,It's not the case that if the old man ever fed...,The old man has fed a snow leopard.,N,type3_again,{'content': ' Does the premise entail ...,[{'content': ' Consider the truth valu...,N
